In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from PopSynthesis.Methods.GNN_activity.utils import visualize_pyg_graph_with_zones, save_graphs
from PopSynthesis.Methods.GNN_activity.graph_construct import construct_graph_pyg, assign_start_time_to_travel_diaries

In [3]:
# Reload the datasets
zones_df = pd.read_csv("data/zones.csv")
purposes_df = pd.read_csv("data/purposes.csv")
sample_households_df = pd.read_csv("data/sample_households.csv")
sample_people_df = pd.read_csv("data/sample_people.csv")
od_matrix_df = pd.read_csv("data/od_matrix.csv")
sample_travel_diaries_df = pd.read_csv("data/sample_travel_diaries.csv")

In [4]:
sample_travel_diaries_df = assign_start_time_to_travel_diaries(sample_travel_diaries_df)

In [5]:
relationships = ["Parent", "Child", "Spouse", "Housemate", "Sibling"]
relationship_mapping = {"Parent": 0, "Child": 1, "Spouse": 2, "Housemate": 3, "Sibling": 4}

# Construct the PyG-compatible heterogeneous graph (CPU mode only)
train_graph_pyg = construct_graph_pyg(zones_df, purposes_df, sample_households_df, sample_people_df, od_matrix_df, sample_travel_diaries_df, relationship_mapping)

# Display the graph structure
train_graph_pyg

HeteroData(
  zone={ x=[5, 1] },
  purpose={ x=[30, 1] },
  household={ x=[7, 2] },
  person={ x=[21, 2] },
  (zone, connects, zone)={
    edge_index=[2, 10],
    edge_attr=[10, 2],
    t=[10],
  },
  (zone, has_household, household)={
    edge_index=[2, 7],
    t=[7],
  },
  (household, has_person, person)={
    edge_index=[2, 21],
    t=[21],
  },
  (zone, has_purpose, purpose)={
    edge_index=[2, 30],
    t=[30],
  },
  (purpose, attracts, person)={
    edge_index=[2, 83],
    t=[83],
    joint_activity=[83],
  },
  (person, relate, person)={
    edge_index=[2, 46],
    relationship=[46],
    t=[46],
  }
)

In [ ]:
# Generate an interactive PyVis visualization with Zones, Households, People, and Purposes correctly labeled
net_visual_zones = visualize_pyg_graph_with_zones(train_graph_pyg, sample_people_df, sample_households_df, purposes_df, zones_df)
net_visual_zones.save_graph("data/graph_starting.html")

In [7]:
save_graphs(train_graph_pyg, output_dir="data/", output_name="train_graph", output_type="torch")

In [8]:
to_predict_hh_df = pd.read_csv("data/to_predict_households.csv")
to_predict_people_df = pd.read_csv("data/to_predict_people.csv")

In [9]:
to_predict_graphs = construct_graph_pyg(zones_df, purposes_df, to_predict_hh_df, to_predict_people_df, od_matrix_df, None, relationship_mapping)
to_predict_graphs

HeteroData(
  zone={ x=[5, 1] },
  purpose={ x=[30, 1] },
  household={ x=[16, 2] },
  person={ x=[52, 2] },
  (zone, connects, zone)={
    edge_index=[2, 10],
    edge_attr=[10, 2],
    t=[10],
  },
  (zone, has_household, household)={
    edge_index=[2, 16],
    t=[16],
  },
  (household, has_person, person)={
    edge_index=[2, 52],
    t=[52],
  },
  (zone, has_purpose, purpose)={
    edge_index=[2, 30],
    t=[30],
  },
  (purpose, attracts, person)={
    edge_index=[2, 52],
    t=[52],
    joint_activity=[52],
  },
  (person, relate, person)={
    edge_index=[2, 132],
    relationship=[132],
    t=[132],
  }
)

In [10]:
save_graphs(to_predict_graphs, output_dir="data/", output_name="to_predict_graph", output_type="torch")

In [11]:
# Generate an interactive PyVis visualization with Zones, Households, People, and Purposes correctly labeled
net_visual_predict = visualize_pyg_graph_with_zones(to_predict_graphs, to_predict_people_df, to_predict_hh_df, purposes_df, zones_df)
net_visual_predict.save_graph("data/graph_predict.html")